In [1]:
import sys
import pathlib

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Local
cwd = pathlib.Path().resolve()
src = cwd.parent
root = src.parent
sys.path.append(str(src))
sys.path.append(str(root))

Load in all of the error data

In [4]:
error_MLP_without_data_augmentation = ...
error_MLP_with_data_augmentation = ...

In [3]:
error_CNN_PIL = ...
error_CNN_PID = ...
error_CNN_PIDPID = ...

In [ ]:
error_unet1 = ...
# etc